# Manual h1, h2 thresholds adjustment

Adjust automatically generated threshold by hand based on validation set score.
Initial thresholds is generated by generate_default_thresholds_2level_classifier.ipynb.

## 1. Set up

In [ ]:
import os
import json
import glob
import scipy
import numpy as np

import pandas as pd
import glob

In [ ]:
# set root as ../
import sys
sys.path.append("../")

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

In [ ]:
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
DATA_DIR = ""

# BASE_DATAIR= os.path.normpath(os.path.join(DATA_DIR, ''))

# Use this for get catkeys.
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))
CATEGORY_DICT_CSV=os.path.join(BASE_DATA_DIR, "178_dict.csv")

### Virtual Category case

In [ ]:
CATSTORE_PATH = ""

In [ ]:
from category import VirtualCategories
vc = VirtualCategories.from_file(CATSTORE_PATH)
catkeys = vc.keys()

### 178 category case

In [ ]:
from category import load_category_dict

In [ ]:
catdict = load_category_dict(CATEGORY_DICT_CSV)

### Load first score df, second score df

In [ ]:
#created by classifier_similarity.ipynb
VALID_DF_PATH=""

VALID_DISTANCE_DICT_PATH=""

#created by generate_default_thresholds_2level_classifier .ipynb
VALID_SECSCORE_PATH=""

In [ ]:
first_scoredf = pd.read_pickle(VALID_DF_PATH)

In [ ]:
second_scoredf = pd.read_pickle(VALID_SECSCORE_PATH)

In [ ]:
distdf = pd.read_pickle(VALID_DISTANCE_DICT_PATH)

In [ ]:
df = first_scoredf
# df = second_scoredf

# Evaluate df

In [ ]:
from visualize import plot_image_list

In [ ]:
def plot_cat(df, cat, thrld, origin=0):
    filtered = df[df[cat] > thrld].sort_values(by=cat, ascending=False)
    for i in range(0, 50, 5):
        print(",".join([os.path.basename(os.path.dirname(v)) for v in filtered['filepaths'].values[i+origin:(i+5+origin)]]))
    for i in range(0, 50, 5):
        print(",".join(["{0:.4f}".format(v) for v in filtered[cat].values[origin+i:(origin+5+i)]]))
    plot_image_list(filtered["filepaths"].values[origin:(origin+50)])

In [ ]:
TRAIN_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "train"))

In [ ]:
import random

In [ ]:
def plot_valid_cat(catkey):
    valds = list(glob.glob(os.path.normpath("{}/{}/*.jpg").format(VALID_DATA_DIR, catkey)))
    #valds = list(glob.glob(os.path.normpath("{}/{}/*.jpg").format(TRAIN_DATA_DIR, catkey)))
    plot_image_list(random.sample(valds, 20))

In [ ]:
def plot_train_cat(catkey):
    valds = list(glob.glob(os.path.normpath("{}/{}/*.jpg").format(TRAIN_DATA_DIR, catkey)))
    plot_image_list(random.sample(valds, 20))

In [ ]:
# dup from classifier_similarity.ipynb
def listdist15(targetkey):
    dist = distdf[targetkey].sort_values(ascending=False)
    dist = dist[dist > 1.5]
    
    for idx in dist.index:
        print("{0}:   {2:.4f} {1}".format(idx, vc.name(idx), dist[idx]))

In [ ]:
vc.name('69'), vc.name('133')

In [ ]:
def plot_cat2(cat, origin=0):
    plot_cat(df, cat, 0.1, origin)

In [ ]:
# plot_valid_cat('105')
# plot_valid_cat('170')
# plot_valid_cat('2')
# plot_valid_cat('3')
# plot_valid_cat('113') 

In [ ]:
listdist15('33')

In [ ]:
# plot_cat2('43_80', 500)

In [ ]:
# plot_cat2('71', 990)

In [ ]:
# plot_cat2('160', 1130)

In [ ]:
# plot_valid_cat('21')

### Category selection 

In [ ]:
catproc_res =[
]

In [ ]:
notyet = [tup for tup in allcatproc if tup[0] not in catset]
notyet

In [ ]:
len(catproc_res)

In [ ]:
# plot_cat(scoredf, "76", 0.1, origin=0)

In [ ]:
# plot_cat(scoredf, '160', 0.1, 2700)

In [ ]:
# plot_cat(df, '65_81', 0.1, 400)

In [ ]:
'11_13_54_120_130_58_86_96'

In [ ]:
catdict[93]

In [ ]:
# plot_cat(scoredf, '93', 0.94, 0)

In [ ]:
# plot_cat(df, '20', 0.8, 100)

In [ ]:
# plot_cat(scoredf, '47_73', 0.94, 0)

In [ ]:
targetkey = 67
print(catdict[targetkey])
# plot_valid_cat(str(targetkey))

In [ ]:
# plot_cat(scoredf, str(targetkey), 0.1, 0)

In [ ]:
catdict[141]

In [ ]:
#plot_train_cat('155')

In [ ]:
#plot_cat(scoredf, "141", 0.1, 100)

In [ ]:
# plot_cat(df, '177', 0.1, 0)

In [ ]:
# plot_cat(df, '47', 0.1, 0)

In [ ]:
#plot_valid_cat('142')

In [ ]:
# plot_valid_cat('65')

In [ ]:
# plot_valid_cat('54')

In [ ]:
# plot_valid_cat('93')

In [ ]:
plot_valid_cat()

In [ ]:
name2id = {catdict[key]:key for key in catdict.keys()}

In [ ]:
name2id['']

In [ ]:
# plot_valid_cat('124')

In [ ]:
name2id['']

In [ ]:
plot_valid_cat('124')

In [ ]:
name2id['']

In [ ]:
plot_valid_cat('124')

### Find bad category (Top 40 precision)

Some category seems fail to train.
From observation, it seems that some category fail to train when prediction result of top 40 scores is missclassified too much.
So I create score for top 40 missclassified rate.

In [ ]:
def calc_top40score(targetkey):
    top40 = scoredf.sort_values(by=targetkey, ascending=False)[0:40]
    return sum(top40['label'] == targetkey)/40

In [ ]:
list(vc.keys())[0]

In [ ]:
calc_top40score('70')

In [ ]:
catkeys = sorted(list(catdict.keys()))

In [ ]:
res = [(key, calc_top40score(str(key)), catdict[key]) for key in catkeys]

In [ ]:
catkeys = sorted(list(vc.keys()))

In [ ]:
res = [(key, calc_top40score(key), vc.name(key)) for key in catkeys]

In [ ]:
from operator import itemgetter

In [ ]:
sortedres = sorted(res, key=itemgetter(1))

In [ ]:
sortedres

In [ ]:
sortedres

In [ ]:
targetkey = '0'

In [ ]:
top40 = scoredf.sort_values(by=targetkey, ascending=False)[0:40]

In [ ]:
sum(top40['label'] == targetkey)/40

In [ ]:
# "49
retraining = ["38", "41", "51", "55", "60", "62", "64", "66", "67", "70", "71", "72", "74", "111", "115", "117", "118", "121", "125", "126", "127", "129"]

In [ ]:
str(sortedres[6][0]) in retraining

In [ ]:
[tup for tup in sortedres if str(tup[0]) in retraining]

In [ ]:
len([])

In [ ]:
from visualize import plot_image_list

In [ ]:
targetkey='14'

In [ ]:
catdict[14], catdict[35]

In [ ]:
catdict[38]

In [ ]:
# plot_cat(scoredf, '38', 0.1)

In [ ]:
# listdist15('4')

In [ ]:
# plot_cat(scoredf, '4', 0.1, 2550)

In [ ]:
[catdict[int(key)] for key in '11_13_54_120_130_58_86_96'.split('_')]

### Investigation

In [ ]:
from visualize import plot_image_list

In [ ]:
scoredf_org = pd.read_pickle("")

In [ ]:
sorted_org = scoredf_org.sort_values(by='65', ascending=False)

In [ ]:
# plot_image_list(sorted_org[sorted_org['65'] > 0.8]['filepaths'].values[0:40])

In [ ]:
sorteddf = scoredf.sort_values(by='65', ascending=False)

In [ ]:
#plot_image_list(sorteddf[sorteddf['65'] > 0.8]['filepaths'].values[0:40])

In [ ]:
from IPython.display import Image, display

In [ ]:
# broken image check
display(Image(''))

In [ ]:
calc_raw_scores('65',  0.86)

In [ ]:
sum(scoredf['65'] > 0.86)

In [ ]:
targetkey = '106'

In [ ]:
thrds =  [val*0.01 for val in range(50, 100)] 

In [ ]:
raw_scores = [calc_raw_scores(targetkey, thrd) for thrd in thrds]

In [ ]:
EPS = 0.0001

In [ ]:
scoretups = [(dic["TP"]/(dic["TP"]+dic["FP"]+EPS), dic["TP"]/(dic["TP"]+dic["FN"]+EPS), 2*dic["TP"]/(2*dic["TP"]+dic["FP"]+dic["FN"]+EPS)) for dic in raw_scores]

In [ ]:
f1scores = pd.DataFrame({"precision":[tup[0] for tup in scoretups], "recall":[tup[1] for tup in scoretups], "f1":[tup[2] for tup in scoretups]})

In [ ]:
f1scores.index = thrds

In [ ]:
f1scores["f1"].idxmax()

In [ ]:
f1scores.plot()

In [ ]:
f1scores.plot()

In [ ]:
similarkeyset = set(distdf[targetkey][distdf[targetkey] >= DIST_THRESHOLD].index)

In [ ]:
similarkeyset

In [ ]:
[catdict[int(key)] for key in similarkeyset]

In [ ]:
sum(scoredf['label'] == '65')

In [ ]:
scoredf[scoredf['label'] == '65']['65']

In [ ]:
wrongs = set(scoredf[scoredf['65'] > 0.83]['label'].values)

In [ ]:
[catdict[int(key)] for key in wrongs]

In [ ]:
wrongs

In [ ]:
sum(scoredf[scoredf['65'] > 0.83]['label'] == '87')

In [ ]:
catdict[81]

In [ ]:
simage = set(distdf['43'][distdf['43'] >= DIST_THRESHOLD].index)

In [ ]:
distdf['43'][distdf['43'] >= DIST_THRESHOLD]

In [ ]:
len(scoredf[scoredf['65'] > 0.7]['label'])

In [ ]:
sum(scoredf[scoredf['65'] > 0.5]['label'] == '65')

In [ ]:
len(scoredf[scoredf['65'] > 0.5])